# Sorting and grouping. SQL
#### В этом ноутбуке собрала решения, выводы результатов и свои пояснения к решениям, сами скрипты .sql поместила в папку Scripts в этой же директории

In [1]:
#импорты и подключение
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2

dbname = "WB"
user = "postgres"
password = "root"
host = "localhost"
port = "5432"


engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')
# Функция для выполнения запроса
def execute_query(query):
    with engine.connect() as connection:
        result = connection.execute(text(query))
        return pd.DataFrame(result.fetchall(), columns=result.keys())

# Часть 1
### №1
Для каждого города выведите число покупателей из соответствующей таблицы, сгруппированных по возрастным категориям и отсортированных по убыванию количества покупателей в каждой категории.

Примечание: под возрастной категорией подразумевается возраст человека в полных годах (например, 21, 35, 65 и т.д.). Можете дополнительно написать запрос именно для “категорий”: от 0 до 20 (категория young), от 21 до 49 (категория adult), от 50 и выше (категория old)

In [3]:
#Запрос
query_1 = '''
select city,
case
	when age>0 and age<21 then 'young'
	when age between 21 and 49 then 'adult'
	when age between 50 and 112 then 'old'
	else 'undefined' 
end as "category",
count(*) as "quantity"
from users
group by 1, 2
order by 3 desc, 1
'''

### Пояснение к решению
1. Возраст 0 не учитываем в категории "young", так как им могут быть заполнены пробелы в данных. Самому старому человеку в мире на момент выполнения задания 112 лет, поэтому все значения, <0 и >=112 отнесла в категорию "undefined"
2. Записи были отсортированы по количеству записей в категории по убыванию, после по возрастанию названия города

In [6]:
#Ответ
df1_1 = execute_query(query_1)
df1_1

,city,category,quantity
0,Spokane,old,2
1,Allenport,old,1
2,Altamont,old,1
3,Applegate,adult,1
4,Ashton,old,1
...,...,...,...
94,Whiteside,old,1
95,Wilkes Barre,adult,1
96,Woodman,adult,1
97,Worthington,old,1


# Чаcть 1
### №2
Рассчитайте среднюю цену категорий товаров в таблице products, в названиях товаров которых присутствуют слова «hair» или «home». Среднюю цену округлите до двух знаков после запятой. Столбец с полученным значением назовите avg_price.

Поля в результирующей таблице: avg_price, category.

In [8]:
#Запрос

query_2 = '''
select category, round(avg(price),2) as avg_price
from products
where (name ilike '%hair%' or name ilike '%home%')
group by category
'''

In [10]:
#Ответ

df1_2 = execute_query(query_2)
df1_2

,category,avg_price
0,Home,101.00
1,Beauty,124.00


# Часть 2
### №1
Назовем “успешными” (’rich’) селлерами тех:

кто продает более одной категории товаров
и чья суммарная выручка превышает 50 000
Остальные селлеры (продают более одной категории, но чья суммарная выручка менее 50 000) будут обозначаться как ‘poor’. Выведите для каждого продавца количество категорий, средний рейтинг его категорий, суммарную выручку, а также метку ‘poor’ или ‘rich’.

Назовите поля: seller_id, total_categ, avg_rating, total_revenue, seller_type.Выведите ответ по возрастанию id селлера.

Примечание: Категория “Bedding” не должна учитываться в расчетах.

In [12]:
#Запрос

query_3 = '''with more_one_category as(
select seller_id, count(distinct category) as total_categ, round(avg(rating),2) as avg_rating,
sum(revenue) as total_revenue from sellers
where category!='Bedding'
group by seller_id
having count(distinct category)>1
)
select seller_id, total_categ, avg_rating, total_revenue,
case 
	when total_revenue>50000 then 'rich'
	else 'poor'
end as seller_type
from more_one_category
order by seller_id'''

### Пояснение к решению
1. Сначала в СТЕ отбраем продавцов, которые продают более одной уникальной категории, за исключением Bedding - продавцы, которые учитываются при дальнейшних расчетах
2. Хотя в категориях учитывались только уникальные, в выручке подсчет идет по всем записям продавца, так как это показалось логичнее: в случае дублей непонятно, какую из выручек оставлять

In [15]:
#Ответ
df2_1 = execute_query(query_3)
df2_1

,seller_id,total_categ,avg_rating,total_revenue,seller_type
0,0,2,5.50,87790,rich
1,2,6,3.00,213564,rich
2,5,7,4.50,250719,rich
3,6,3,1.80,150053,rich
4,7,2,4.00,49197,poor
...,...,...,...,...,...
229,252,4,5.60,100718,rich
230,253,5,3.00,135570,rich
231,254,6,4.14,238306,rich
232,255,4,5.25,142591,rich


# Часть 2
### №2
Для каждого из неуспешных продавцов (из предыдущего задания) посчитайте, сколько полных месяцев прошло с даты регистрации продавца.

Отсчитывайте от того времени, когда вы выполняете задание. Считайте, что в месяце 30 дней. Например, для 61 дня полных месяцев будет 2.

Также выведите разницу между максимальным и минимальным сроком доставки среди неуспешных продавцов. Это число должно быть одинаковым для всех неуспешных продавцов.

Назовите поля: seller_id, month_from_registration ,max_delivery_difference.Выведите ответ по возрастанию id селлера.

Примечание: Категория “Bedding” по-прежнему не должна учитываться в расчетах.

In [17]:
#Запрос

query_4 = '''with unsuccess as(
select seller_id,
(current_date-min(date_reg))/30 as month_from_registration,
min(delivery_days) as min_dd,
max(delivery_days) as max_dd
from sellers
where category!='Bedding'
group by seller_id
having not(count(distinct category)>1 and sum(revenue)>50000)
)
select seller_id,
month_from_registration,
(select max(max_dd) from unsuccess)-(select min(min_dd) from unsuccess) as max_delivery_difference
from unsuccess
order by seller_id'''

### Пояснение к решению
1. В предыдущем задании как успешные были определены продавцы из категории 'rich', соответственно, неуспешными являются все оставшиеся(даже те, кто продает только 1 категорию товаров)
2. В качестве даты регистрации была выбрана наименьшая
3. Для каждого продавца отобрали наименьший и наибольший срок доставки. Впоследствии разница между максимальным из наибольших сроков и минимальным из наименьших выводилась для каждого продавца(для всех одинаковая, как и сказано в условии)

In [19]:
#Ответ
df2_2 = execute_query(query_4)
df2_2.head(10) #первые 10 записей
#df2_2

,seller_id,month_from_registration,max_delivery_difference
0,1,84,9
1,4,68,9
2,7,154,9
3,8,82,9
4,9,110,9
5,20,103,9
6,25,60,9
7,29,171,9
8,44,113,9
9,46,156,9


# Часть 2
### №3. Версия 1
Отберите продавцов, зарегистрированных в 2022 году и продающих ровно 2 категории товаров с суммарной выручкой, превышающей 75 000.

Выведите seller_id данных продавцов, а также столбец category_pair с наименованиями категорий, которые продают данные селлеры.

Например, если селлер продает товары категорий “Game”, “Fitness”, то для него необходимо вывести пару категорий category_pair с разделителем “-” в алфавитном порядке (т.е. “Game - Fitness”).

Поля в результирующей таблице: seller_id, category_pair

In [21]:
#Запрос

query_5 = '''WITH seller_categories AS (
    SELECT
        seller_id,
        ARRAY_AGG(DISTINCT category ORDER BY category) AS categories,
        SUM(revenue) AS total_revenue
    FROM
        sellers
    WHERE
        EXTRACT(YEAR FROM date_reg) = 2022
    GROUP BY
        seller_id
    HAVING
        COUNT(DISTINCT category) = 2
        AND SUM(revenue) > 75000
)
SELECT
    seller_id,
    CONCAT(categories[1], ' - ', categories[2]) AS category_pair
FROM
    seller_categories
ORDER BY
    seller_id'''

### Пояснение к решению
1. В качестве количества категорий было выбрано количество уникальных категорий 
2. В этой версии решения проверялись все даты регистрации одного продавца

In [24]:
#Ответ
df2_3 = execute_query(query_5)
df2_3

,seller_id,category_pair
0,108,Camera - Camping
1,132,Cooking - Outdoors
2,168,Camera - Truck
3,207,Jewelry - Movie


### №3. Версия 2

In [26]:
query_6='''
WITH seller_categories AS (
    SELECT
        seller_id,
        ARRAY_AGG(DISTINCT category ORDER BY category) AS categories,
        SUM(revenue) AS total_revenue
    FROM
        sellers  
    GROUP BY
        seller_id
    HAVING
        COUNT(DISTINCT category) = 2
        AND SUM(revenue) > 75000 AND
        EXTRACT(YEAR FROM min(date_reg)) = 2022
)
SELECT
    seller_id,
    CONCAT(categories[1], ' - ', categories[2]) AS category_pair
FROM
    seller_categories
ORDER BY
    seller_id
'''

### Пояснение к решению
1. В этой версии решения проверялась только минимальная дата регистрации одного продавца

In [28]:
#Ответ - таких продавцов нет
df2_3_2 = execute_query(query_6)
df2_3_2

,seller_id,category_pair
